In [6]:
# K-Means
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
from sklearn.cluster import KMeans
from utils import *
import random
import numpy as np
random.seed(49)
np.random.seed(49)

# 参数设置 Parameter Set
data_Folder = "Simulated_data\\"
K           = 3          # 簇数 Num of cluster
train_ratio = 0.72
val_ratio   = 0.18
save_path   = "Process_1\\K-means\\"
os.makedirs(save_path,exist_ok=True)
temp_file   = "beijing_temp_halfhour_99.npy"
user_num    = 150
# data_idx    = 1

for data_idx in range(1,11):
    load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
    temp_data   = np.load(data_Folder+temp_file) #加载温度数据 temperaure data
    time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index
    kmeans      = KMeans(n_clusters=K,random_state=49,n_init='auto')
    k_data      = load_data[:,:int(load_data.shape[1]*train_ratio)]
    kmeans.fit(k_data)
    cluster_list = np.array(kmeans.labels_)
    aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='sum')
    model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                    lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
    
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                        val_ratio=val_ratio,temp_data=temp_data,
                                        time_index=time_index,model_dict=model_dict)
    
    predict_result = predict_result.sum(axis=0)
    test_result = test_result.sum(axis=0)
    
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",predict_result)
    np.save(save_path+"test_result_"+str(data_idx)+".npy",test_result)

In [7]:
# GMM
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import random
import numpy as np
from sklearn.mixture import GaussianMixture
from utils import *

random.seed(49)
np.random.seed(49)

# 参数设置 Parameter Set
data_Folder = "simulated_data\\"
K           = 3          # 簇数 Num of cluster
train_ratio = 0.72
val_ratio   = 0.18
save_path   = "Process_1\\GMM\\"
os.makedirs(save_path,exist_ok=True)

for data_idx in range(1,11):
    gmm = GaussianMixture(n_components=3, covariance_type="diag", n_init=10, random_state=49)
    load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
    temp_data   = np.load(data_Folder+temp_file) #加载温度数据 temperaure data
    time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index

    gmm.fit(load_data[:,:int(load_data.shape[1]*train_ratio)])
    cluster_list = np.array(gmm.predict(load_data[:,:int(load_data.shape[1]*train_ratio)]))
    aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='sum')
    model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                    lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                        val_ratio=val_ratio,temp_data=temp_data,
                                        time_index=time_index,model_dict=model_dict)
    predict_result = predict_result.sum(axis=0)
    test_result = test_result.sum(axis=0)
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",predict_result)
    np.save(save_path+"test_result_"+str(data_idx)+".npy",test_result)

In [8]:
# UP_to_Down
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import random
import numpy as np
from sklearn.cluster import KMeans
from utils import *

random.seed(49)
np.random.seed(49)

# 参数设置 Parameter Set
data_Folder = "simulated_data\\"
K           = 1          # 簇数 Num of cluster
train_ratio = 0.72
val_ratio   = 0.18
save_path   = "Process_1\\Up_to_Down\\"
os.makedirs(save_path,exist_ok=True)

# data_idx    = 1

for data_idx in range(1,11):
    kmeans      = KMeans(n_clusters=K,random_state=49,n_init='auto')
    load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
    temp_data   = np.load(data_Folder+temp_file) #加载温度数据 temperaure data
    time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index

    kmeans.fit(load_data[:,:int(load_data.shape[1]*train_ratio)])
    cluster_list = np.array(kmeans.labels_)
    aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='sum')
    model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                    lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
    os.makedirs(save_path,exist_ok=True)
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                        val_ratio=val_ratio,temp_data=temp_data,
                                        time_index=time_index,model_dict=model_dict)
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",predict_result)
    np.save(save_path+"test_result_"+str(data_idx)+".npy",test_result)

In [9]:
# Bottom_to_Up
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import random
import numpy as np
from sklearn.cluster import KMeans
from utils import *

random.seed(49)
np.random.seed(49)

# 参数设置 Parameter Set
data_Folder = "simulated_data\\"
K           = user_num          # 簇数 Num of cluster
train_ratio = 0.72
val_ratio   = 0.18
save_path   = "Process_1\\Bottom_to_Up\\"
os.makedirs(save_path,exist_ok=True)

# data_idx    = 1

for data_idx in range(1,11):
    kmeans      = KMeans(n_clusters=K,random_state=49,n_init='auto')
    load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
    temp_data   = np.load(data_Folder+temp_file) #加载温度数据 temperaure data
    time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index

    kmeans.fit(load_data[:,:int(load_data.shape[1]*train_ratio)])
    cluster_list = np.array(kmeans.labels_)
    aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='sum')
    model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                    lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
    os.makedirs(save_path,exist_ok=True)
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                        val_ratio=val_ratio,temp_data=temp_data,
                                        time_index=time_index,model_dict=model_dict)
    predict_result = predict_result.sum(axis=0)
    test_result = test_result.sum(axis=0)
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",predict_result)
    np.save(save_path+"test_result_"+str(data_idx)+".npy",test_result)

In [10]:

# Ensemble
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"

import random
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from utils import *

random.seed(49)
np.random.seed(49)

# 参数设置 Parameter Set
data_Folder = "simulated_data\\"
# K           = 1          # 簇数 Num of cluster
train_ratio = 0.72
val_ratio   = 0.18
save_path   = "Process_1\\Ensemble\\"
os.makedirs(save_path,exist_ok=True)

# data_idx    = 1

for data_idx in range(1,11):
    predict_val_total = []
    test_val_total = []
    predict_result_total = []
    test_result_total = []
    for K in [i for i in range(1,21)]:
        kmeans      = KMeans(n_clusters=K,random_state=49,n_init='auto')
        load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
        temp_data   = np.load(data_Folder+temp_file) #加载温度数据 temperaure data
        time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index

        kmeans.fit(load_data[:,:int(load_data.shape[1]*train_ratio)])
        cluster_list = np.array(kmeans.labels_)
        aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='sum')
        model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                        lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
        
        predict_val, test_val = Predict_val(cluster_num=K,cluster_list=cluster_list,
                                            aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                            val_ratio=val_ratio,temp_data=temp_data,
                                            time_index=time_index,model_dict=model_dict)
        predict_val_total.append(predict_val.sum(axis=0))
        test_val_total.append(test_val.sum(axis=0))
        
        predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                            aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                            val_ratio=val_ratio,temp_data=temp_data,
                                            time_index=time_index,model_dict=model_dict)
        predict_result_total.append(predict_result.sum(axis=0))
        test_result_total.append(test_result.sum(axis=0))
    
    predict_val_total = np.asarray(predict_val_total).T
    test_val_total = np.asarray(test_val_total).mean(axis=0).T
    
    Combinator = LinearRegression().fit(X=predict_val_total,y=test_val_total)
    predict_result_total = Combinator.predict(np.asarray(predict_result_total).T)
    
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",np.asarray(predict_result_total))
    np.save(save_path+"test_result_"+str(data_idx)+".npy",np.asarray(test_result_total).mean(axis=0))